In [ ]:
import requests

import pandas as pd

from serlo.mysql import db
#import mysql.connector as mysql
MAILCHIMP_API_KEY = ""
dc = MAILCHIMP_API_KEY.split('-')[1]

url = f"https://{dc}.api.mailchimp.com/3.0/lists/a7bb2bbc4f/members"

In [ ]:
df = pd.read_sql("""
    Select
        user.id,
        user.username,
        user.description,
        user.date,
        sum(events.edits) as edits,
        sum(events.reviews) as reviews,
        sum(events.comments) as comments,
        sum(events.taxonomy) as taxonomy
    From (
        SELECT
            IF(event_log.event_id = 5, 1, 0) as edits,
            IF(event_log.event_id in (6,11), 1, 0) as reviews,
            IF(event_log.event_id in (8,9,14,16), 1, 0) as comments,
            IF(event_log.event_id in (1,2,12,15,17), 1, 0) as taxonomy,
            event_log.actor_id
        FROM event_log
        WHERE event_log.event_id IN (5,6,11,8,9,14,16,1,2,12,15,17)
    ) events
    right join user on user.id = events.actor_id
    group by user.id
""", db)
df.fillna(0, inplace=True)

df["date"] = df["date"].map(pd.to_datetime)
inactive_users = df[
    (df["edits"] + df["reviews"] + df["taxonomy"] == 0) &
    (df["description"] != "NULL") & (df["description"] != "") &
    (df["date"] < pd.to_datetime("2021-12-01"))
].copy()
inactive_users.sort_values(["date"], inplace=True)

In [ ]:
subscribers = requests.get(url, params = {"apikey": MAILCHIMP_API_KEY, "count": 14000})
subscribers.json()

In [ ]:
subscribers = requests.get(url, params = {"apikey": MAILCHIMP_API_KEY, "status": "subscribed", "fields": "total_items"})
subscribers.json()

In [ ]:
subscribers.reason

In [ ]:
act_url = f"https://{dc}.api.mailchimp.com/3.0/lists/a7bb2bbc4f"
activity = requests.get(act_url, params = {"apikey": MAILCHIMP_API_KEY})
activity.json()

In [ ]:
is_possible_fake_account(subscriber):
    if !(subscriber in inactive_users) || subscriber.stats.avg_open_rate > 0
        return false
    return true
    

In [ ]:
filtered = filter(lambda subscriber: is_possible_fake_account(subscriber))
filtered.id

In [ ]:
### You can check the profile of the user here: serlo.org/{id}

In [ ]:
delete_url = "/lists/a7bb2bbc4f/members/{subscriber_hash}/actions/delete-permanent

delete_from_newsletter():
    requests.post(delete_url, key: MAILCHIMP_API_KEY)